In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from numpy.random import seed
from numpy.random import randint
import random
from math import sqrt
import string
import sys

### Read CSV
##### Input: filename

In [142]:
def read_file(name):
    df = pd.read_csv(name);
    # node names array
    namesArr = df.columns.values
    # nodes dictionary
    dict = {}
    for i in range(0, df.columns.size):
        dict[df.columns[i]] = i;
    return namesArr, dict, df.values

### Generate n nodes
##### Input: n (number of nodes), lim - space limit

In [143]:
def generate_n_nodes(n,lim): #For n <= 27
    # Nodes position (x,y)
    seed(1)
    coord = np.zeros((n, 2))
    xList = np.zeros(n)
    yList = np.zeros(n)
    for i in range(0, n):
        x = randint(0, lim)
        xList[i] = x
        y = randint(0, lim)
        yList[i] = y
        coord[i] = (x, y)
    mx = np.max(np.max(coord, axis = 1))
    # Nodes Dictionary
    dict = {}
    alphabet_string = string.ascii_uppercase
    alphabet_list = list(alphabet_string)
    alphabet_list = alphabet_list[:n]
    for i, txt in enumerate(alphabet_list):
        dict[txt] = i;
    # Distances matrix 
    tempList, c = np.zeros(len(coord)**2), 0
    for x,y in coord:
        for i,j in coord :
            tempList[c] = round(sqrt(pow((x-i),2) + pow((y-j),2) ),1)
            c += 1
    mat = tempList.reshape((len(coord), len(coord)))
    dist_mat = pd.DataFrame(mat)
    return coord, dict, dist_mat

### Objective Function
##### Input: nodes names array, nodes dictionary, distances matrix, individual array

In [144]:
def objective_function(namesArr,d, dMatrix, sol):
    if sol[0] != sol[len(sol)-1]:
        print("The first and last character must be the same ")
        return -1 #the fist and last character must be the same
    rep = sol[:len(sol)-1]
    if not len(set(rep)) == len(rep):
        print("the nodes are repeated")
        return -1 # the nodes are repeated
    if sorted(rep) != sorted(namesArr): 
        print("NOT all the nodes are included in the solution")
        return -1 # not all the nodes are included in the solution
    
    n = len(sol)-1
    w = 0;
    for i in range(0, n):
            w+=dMatrix[ord(sol[i])-65][ord(sol[i+1])-65]
    return w

In [145]:
# MAIN
print("\n--------------------------------- 1. Read the source file ------------------------------------------------")
print("\t~~ Nodes names array ~~ ")
print(read_file("adjacency_matrix.csv")[0])
print("\t~~ Nodes dictionary ~~ ")
print(read_file("adjacency_matrix.csv")[1])
print("\~~ Distances Matrix ~~ ")
print(read_file("adjacency_matrix.csv")[2])
print("\n--------------------------------- 2. Generate n nodes ------------------------------------------------")
print("\t~~ Nodes position (x,y) ~~ ")
n = 10
lim = 100
print(generate_n_nodes(n,lim)[0] )
print("\t~~ Nodes dictionary ~~ ")
print(generate_n_nodes(n,lim)[1] )
print("\t~~ Distances Matrix ~~ ")
print(generate_n_nodes(n,lim)[2] )
print("\n--------------------------------- 3. Objective function ------------------------------------------------")
namesArr = read_file("adjacency_matrix.csv")[0]
dictionary = read_file("adjacency_matrix.csv")[1]
matrix = read_file("adjacency_matrix.csv")[2]
print("weight: ",objective_function(namesArr, dictionary, matrix, solution))


--------------------------------- 1. Read the source file ------------------------------------------------
	~~ Nodes names array ~~ 
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
	~~ Nodes dictionary ~~ 
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9}
\~~ Distances Matrix ~~ 
[[ 0 59 48 69 54 91 55 34 81 72]
 [59  0 45 19 45 73 48 13 13 42]
 [48 45  0 97 11 65 12 58 70 45]
 [69 19 97  0 31 94 23 66 71 13]
 [54 45 11 31  0 95 37 47 24 49]
 [91 73 65 94 95  0 62 35 38 61]
 [55 48 12 23 37 62  0 82 96 47]
 [34 13 58 66 47 35 82  0 10 47]
 [81 13 70 71 24 38 96 10  0 63]
 [72 42 45 13 49 61 47 47 63  0]]

--------------------------------- 2. Generate n nodes ------------------------------------------------
	~~ Nodes position (x,y) ~~ 
[[37. 12.]
 [72.  9.]
 [75.  5.]
 [79. 64.]
 [16.  1.]
 [76. 71.]
 [ 6. 25.]
 [50. 20.]
 [18. 84.]
 [11. 28.]]
	~~ Nodes dictionary ~~ 
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9}
	~~ Distance

# Genetic Algorithm

In [146]:
def indices(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

def combine(one, two):
    mid1 = ""
    mid2 = ""
    lenone = int(len(one)/2)
    for i in range(lenone):
        mid1 += one[i]
    for i in range(lenone):
        mid2 += two[i]
    mix = mid1+mid2
    # mix = mix[:-1] + mix[0] #first element = last element
    # look for repeated elements
    dupl = []
    uniqueDulp = []
    for i in range(len(list(mix))):
        if( len(indices(list(mix), mix[i])) > 1 ):
            #print(mix[i], indices(list(mix), mix[i]))
            dupl.append(indices(list(mix), mix[i]))
    # missing elements
    for i in range(len(dupl)):
        for j in range(1,len(dupl[i])):
            uniqueDulp.append( (dupl[i])[j] ) 
    uniqueDupl = np.unique(uniqueDulp)
    
    mix += mix[0] #first element = last element
    sortOne = sorted(one[:-1])
    sortMix = sorted(mix[:-1])
    miss = ""
    flag = False
    for i in range(len(sortOne)):
        for j in range(len(sortMix)):
            if(sortOne[i] == sortMix[j]):
                flag = True
                break
        if(flag == False):
            miss += sortOne[i]
        else: flag = False
    miss = list(miss)
    print(miss)
    mix = list(mix)    
    for i in range(len(uniqueDupl)):
        mix[uniqueDupl[i]] = miss[i]
    return ''.join(mix)   
    
def tournament_selection(parents):
    parDict = {}
    for i in range(len(parents)):
        sol = ''.join(parents[i])
        parDict[sol] =  objective_function(parents[i][:len(parents[i])-1], dictionary, matrix, parents[i])
    parDict = sorted(parDict.items(), key=lambda x: x[1])
    
    one = list(parDict)[0] # select the top 2 best elements
    two = list(parDict)[1] # select the top 2 best elements
    #return combine(one[0],two[0])
    return one, two

In [256]:
def crossover(parents, pr): # pr = probability of reproduction
    gen = []
    for i in range(len(parents)):
        if(random.uniform(0, 1) < pr):
            print("select two")
            # tournament
            first = tournament_selection(parents)[0]
            second = tournament_selection(parents)[1]
            gen.append(list(combine(first[0], second[0])))
        else:
            print("clone")
            #tournament
            first =tournament_selection(parents)[0]
            gen.append(list(first[0]))
    return gen

In [257]:
def genetic_algorithm(n, g, pr, pm):
    nNodes = 10
    generate_n_nodes(nNodes, 10) 
    namesArrList = (generate_n_nodes(n, lim)[1]).keys()
    dictionary = generate_n_nodes(n, lim)[1]
    matrix = generate_n_nodes(n, lim)[2]
    #create the initial population
    parents = []
    for i in range(n):
        p = list(np.random.permutation(list(namesArrList)))
        p.append(p[0])
        parents.append(p)
    #Calculate the population fitness & get the elite
    elite = float('inf')
    for p in parents:
        elite = min(elite, objective_function(namesArrList, dictionary, matrix, p))
    print("weight: ",elite)
    while g > 0: # While the number of generations is less than G or we haven’t found a good solution
        nGen = crossover(parents, pr)
        #print(parents)
        print(nGen)
        break
    
lim = 100
n = 10 # population size
g = 3 # max number of generations
pr = 0.8 # reproduction's probability
pm = 0.8 # mutation probability

genetic_algorithm(n,g,pr,pm)

weight:  381.49999999999994
clone
clone
select two
['B', 'I']
select two
['B', 'I']
select two
['B', 'I']
select two
['B', 'I']
select two
['B', 'I']
clone
clone
select two
['B', 'I']
[['G', 'J', 'F', 'H', 'E', 'D', 'B', 'I', 'C', 'A', 'G'], ['G', 'J', 'F', 'H', 'E', 'D', 'B', 'I', 'C', 'A', 'G'], ['G', 'J', 'F', 'H', 'E', 'C', 'B', 'D', 'A', 'I', 'G'], ['G', 'J', 'F', 'H', 'E', 'C', 'B', 'D', 'A', 'I', 'G'], ['G', 'J', 'F', 'H', 'E', 'C', 'B', 'D', 'A', 'I', 'G'], ['G', 'J', 'F', 'H', 'E', 'C', 'B', 'D', 'A', 'I', 'G'], ['G', 'J', 'F', 'H', 'E', 'C', 'B', 'D', 'A', 'I', 'G'], ['G', 'J', 'F', 'H', 'E', 'D', 'B', 'I', 'C', 'A', 'G'], ['G', 'J', 'F', 'H', 'E', 'D', 'B', 'I', 'C', 'A', 'G'], ['G', 'J', 'F', 'H', 'E', 'C', 'B', 'D', 'A', 'I', 'G']]
